 <hr />
 Before starting with the notebook ensure pyspark is installed and working. To install and to find the spark use pip install as shown in the below cells.
<hr />

In [1]:
import findspark

In [2]:
print(findspark.find())
findspark.init()

/opt/spark


<hr />
Create a Spark Session
<hr />

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Pipeline") \
    .master('local[2]') \
    .getOrCreate()

23/01/20 18:48:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


<hr />
Create a Dataframe comprising a sentence, an identification value and a sentiment value (0:negative and 1:positive)
<hr />

In [4]:
training = spark.createDataFrame([
     (0, 'i like apple pie for dessert', 1.0),
     (1, 'i dont drive fast cars', 0.0),
     (2, 'data science is fun', 1.0),
     (3, 'chocolate is not my favorite', 0.0),
     (4, 'my favorite movie is predator', 1.0)],
     ['id', 'text', 'label'])

<hr />
Import the relevant pyspark packages <br>
1. Pipeline : To create a Training and Testing Pipeline <br>
2. Tokenizer : To create tokens from the sentence by converting the input string to lowercase and then splits it by white spaces. <br>
3. HashingTF : To generate features from the tokens by Mapping a sequence of terms to their term frequencies using the hashing trick. <br>
4. Logistic Regression : For training a classifier <br>
<hr />

In [5]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer

<hr />
Initialzie the Estimators and Transformers.
<hr />

In [6]:
tokenizer = Tokenizer(inputCol="text", outputCol="words")
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features")
lr = LogisticRegression(maxIter=10, regParam=0.01, featuresCol='features',labelCol='label')

<hr />
Create a Pipeline.
<hr />

In [7]:
pipeline = Pipeline(stages=[tokenizer, hashingTF, lr])

<hr />
Call the fit function for executing the pipeline and generating the trained model.
<hr />

In [8]:
model = pipeline.fit(training)

23/01/20 18:48:53 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
23/01/20 18:48:53 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


<hr />
Display the Stages of the pipeline.
<hr />

In [9]:
model.stages

[Tokenizer_a4cad21b24f1,
 HashingTF_5b40f4dde215,
 LogisticRegressionModel: uid=LogisticRegression_a1eb7d68fdfd, numClasses=2, numFeatures=262144]

<hr />
Initialize the test data.
<hr />

In [10]:
test = spark.createDataFrame([
     (5, 'I like programming'),
     (6, 'I dont eat grapes')],
     ["id", "text"])

<hr />
Use the pipeline to generate predictions for the test data.
<hr />

In [11]:
prediction = model.transform(test)

<hr />
Display the predictions.
<hr />

In [12]:
prediction.show(truncate=False, vertical=True)

-RECORD 0---------------------------------------------------------------
 id            | 5                                                      
 text          | I like programming                                     
 words         | [i, like, programming]                                 
 features      | (262144,[19036,154517,208258],[1.0,1.0,1.0])           
 rawPrediction | [-1.357800623344739,1.357800623344739]                 
 probability   | [0.2045979911446534,0.7954020088553466]                
 prediction    | 1.0                                                    
-RECORD 1---------------------------------------------------------------
 id            | 6                                                      
 text          | I dont eat grapes                                      
 words         | [i, dont, eat, grapes]                                 
 features      | (262144,[19036,87273,188981,202572],[1.0,1.0,1.0,1.0]) 
 rawPrediction | [0.7366020107563789,-0.73660201075

<hr />
Extract only the prediction value from the output of the pipeline.
<hr />

In [13]:
prediction.select("prediction").toJSON().first()

'{"prediction":1.0}'

<hr />
Stop the Spark Session.
<hr />

In [14]:
spark.stop()